<a href="https://colab.research.google.com/github/karchx/ml_101/blob/main/nlp_polizas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!java --version

openjdk 11.0.28 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [ ]:
!pip install tabula-py[jpype] pdfplumber

In [4]:
import pandas as pd
from tqdm import tqdm
import tabula
import math
import os
import pdfplumber

def make_page_ranges(total_pages, batch_size):
    """
    Devuelve lista de strings como "1-10", "11-20", ... hasta total_pages.
    """
    ranges = []
    for start in range(1, total_pages + 1, batch_size):
        end = min(start + batch_size - 1, total_pages)
        if start == end:
            ranges.append(f"{start}")
        else:
            ranges.append(f"{start}-{end}")
    return ranges

def read_pdf_in_batches(
    pdf_path,
    total_pages,
    batch_size=10,
    java_options=None,
    output_intermediate_dir=None,
    **read_pdf_kwargs
):
    """
    Lee un PDF en batches de páginas y concatena los dataframes resultantes.

    Parámetros clave:
      - pdf_path: ruta al PDF.
      - total_pages: número total de páginas del PDF (puedes calcularlo con PyPDF2 / pdfplumber).
      - batch_size: cuántas páginas agrupar por batch.
      - java_options: opcional, p.ej. "-Xmx2g" para tabula.
      - output_intermediate_dir: si se pasa, guarda cada batch como parquet ahí y luego los concatena.
      - read_pdf_kwargs: cualquier argumento adicional para tabula.read_pdf (e.g., lattice=True/False).
    """
    page_ranges = make_page_ranges(total_pages, batch_size)
    all_dfs = []
    os.makedirs(output_intermediate_dir, exist_ok=True) if output_intermediate_dir else None

    for pr in tqdm(page_ranges, desc="Leyendo batches de PDF"):
        try:
            if java_options:
                os.environ["TABULA_JAVA_OPTIONS"] = java_options
            dfs = tabula.read_pdf(pdf_path, pages=pr, multiple_tables=True, **read_pdf_kwargs)
            if not isinstance(dfs, list):
                # A veces devuelve un solo df
                dfs = [dfs]
        except Exception as e:
            # Loguear y continuar
            print(f"[WARNING] Falló batch {pr}: {e}")
            continue

        if output_intermediate_dir:
            # Guardar cada df individualmente para ese batch
            for i, df in enumerate(dfs):
                fname = os.path.join(output_intermediate_dir, f"batch_{pr.replace('-', '_')}_table_{i}.parquet")
                try:
                    df.to_parquet(fname)
                except Exception as e:
                    print(f"[WARNING] No se pudo escribir {fname}: {e}")
        else:
            all_dfs.extend(dfs)

    # Si se usó persistencia intermedia, recargar todo desde disco
    if output_intermediate_dir:
        parquet_files = sorted(
            [os.path.join(output_intermediate_dir, f) for f in os.listdir(output_intermediate_dir) if f.endswith(".parquet")]
        )
        for pf in tqdm(parquet_files, desc="Recargando parquets intermedios"):
            try:
                df = pd.read_parquet(pf)
                all_dfs.append(df)
            except Exception as e:
                print(f"[WARNING] Error leyendo {pf}: {e}")

    if not all_dfs:
        raise RuntimeError("No se leyó ninguna tabla del PDF.")

    combined = pd.concat(all_dfs, ignore_index=True)
    return combined


pdf_path = "test.pdf"
with pdfplumber.open(pdf_path) as pdf:
    total_pages = len(pdf.pages)

# Leer en batches de 20 páginas, usando stream (ajusta según estructura de tablas)
combined_df = read_pdf_in_batches(
    pdf_path,
    total_pages=total_pages,
    batch_size=20,
    java_options="-Xmx2g",  # si necesitas más heap para Java
    lattice=False,          # o True dependiendo de tu PDF
    stream=True,
    output_intermediate_dir="intermediate_tables"  # opcional: persiste en disco
)

# Save the DataFrame to an Excel file
excel_output_path = "output_excel_file.xlsx"
combined_df.to_excel(excel_output_path, index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'test.pdf'

In [ ]:
!pip install PyMuPDF camelot-py tabula-py

In [ ]:
import fitz  # PyMuPDF
import camelot
import pandas as pd
import typing as T
import re


def extract_tables_from_pdf(pdf_path):
    tables = camelot.read_pdf(pdf_path, flavor='stream', edge_tol=500)
    return tables


def extract_data(pdf_path: str) -> list:
    tables = extract_tables_from_pdf(pdf_path)
    # plt = camelot.plot(tables[2], kind='contour')
    # plt.show()
    df = []
    for table in tables:
        df.append(table.df)

    return df

def purge_list_dict(data: T.List[T.Any]):
    return [item for item in data if item]

def clean_numeric_value(value: str):
  if not isinstance(value, float):
    value = value.replace("Q.", "#")

  cleaned_value = re.sub(r'[^\d.]', '', value)
  parts = cleaned_value.split('.')
  if len(parts) > 1:
    cleaned_value = f"{parts[0]}.{''.join(parts[1:])}"
  else:
    cleaned_value = parts[0]
  return cleaned_value

In [ ]:
def set_group(df: pd.DataFrame, filter: str):
  dict_set = {}
  try:
    for col in df.columns:
      if df[col].str.contains(filter, case=False).any():
        df = df.map(lambda x: np.nan if x == '' else x)
        df = df.dropna(how='all', axis=1)

        key = df.iat[0, df.columns.get_loc(col)]
        value_str = df.iat[0, df.columns.get_loc(col) + 1]

        try:
          value = float(clean_numeric_value(value_str))
          dict_set["key"] = key
          dict_set["value"] = value
        except (ValueError, TypeError):
          continue
        break
  except KeyError as e:
    pass
  return dict_set

In [ ]:
import numpy as np

pdf_path = "./test2.pdf"
dfs = extract_data(pdf_path)

# Loop in all dataframes in return tables `extract_data` method
data = []
for df in dfs:
  df = df.rename(columns=lambda x: f"column_{x+1}")

  prima_row = df[df.apply(lambda row: row.astype(str).str.contains('PRIMA', case=False).any(), axis=1)]
  suma_asegurada_row = df[df.apply(lambda row: row.astype(str).str.contains('SUMA', case=False).any(), axis=1)]
  total_row = df[df.apply(lambda row: row.astype(str).str.contains('TOTAL', case=False).any(), axis=1)]
  iva_row = df[df.apply(lambda row: row.astype(str).str.contains('IVA', case=False).any(), axis=1)]
  gtos_emision_row = df[df.apply(lambda row: row.astype(str).str.contains('GTOS. EMI', case=False).any(), axis=1)]
  gtos_fracc_row = df[df.apply(lambda row: row.astype(str).str.contains('GTOS. FRACC', case=False).any(), axis=1)]
  gtos_otros_row = df[df.apply(lambda row: row.astype(str).str.contains('OTROS', case=False).any(), axis=1)] # check
  dto_row = df[df.apply(lambda row: row.astype(str).str.contains('DTO', case=False).any(), axis=1)]

  data.append(set_group(prima_row, 'PRIMA'))
  data.append(set_group(suma_asegurada_row, 'SUMA'))
  data.append(set_group(total_row, 'TOTAL'))
  data.append(set_group(iva_row, 'IVA'))
  data.append(set_group(gtos_emision_row, 'GTOS. EMI'))
  data.append(set_group(gtos_fracc_row, 'GTOS. FRACC'))
  data.append(set_group(gtos_otros_row, 'OTRO'))
  data.append(set_group(dto_row, 'DTO'))

data = purge_list_dict(data)
df = pd.DataFrame.from_dict(data)

# DEBUG
display(df.drop_duplicates())

,key,value
0,Prima,158400.0
1,Suma Asegurada:,132000000.0
2,Total,189438.4
3,IVA,19958.4
4,Gtos. Emisión,7920.0
5,Gtos. Fracc.,0.0
6,Dto. 1422,3160.0
